# Compute landcover areas

#### Import statements

In [1]:
import grass.script as gscript

In [2]:
from grass.pygrass.vector.geometry import Point
from grass.pygrass.vector import Vector

from grass.pygrass.vector import VectorTopo
from grass.pygrass.vector.table import DBlinks

In [3]:
import numpy as np
import pandas
import pyprind

#### Variable definitions

define buffer radius

In [4]:
radius = 500

#### Function declarations

connect to attribute table

In [5]:
def connectToAttributeTable(map):
    vector = VectorTopo(map)
    vector.open(mode='r')
    dblinks = DBlinks(vector.c_mapinfo)
    link = dblinks[0]
    return link.table()

extract point from vector map

In [6]:
def extractPoint(input, ID, output):

    where = 'ID = {0}'.format(ID)
    type = 'point'

    gscript.read_command('v.extract',
                         input=input,
                         where=where,
                         output=output,
                         type=type, 
                         overwrite=True)

create buffer around point

In [7]:
def bufferPoint(input, output, radius):
    gscript.read_command('v.buffer',
                         input=input,
                         output=output,
                         type='point',
                         distance=radius,
                        overwrite=True)

convert viewshed from raster to vector map

In [8]:
def vectorizeViewshed(input, ID, output):
    type = 'area'
    column = 'visible'

    gscript.read_command('r.to.vect', 
                         input=input, 
                         output=output,
                         type=type,
                         column=column,
                         overwrite=True)

overlay a vector map on an underlying vector map using 'and' selection operator

In [9]:
def overlay(overlay, underlay, output):
    operator='and'
    gscript.read_command('v.overlay',
                         ainput=overlay,
                         binput=underlay,
                         operator=operator,
                         output=output,
                         overwrite=True)

add area column to vector map

In [10]:
def calculateAreas(map):
    
    #add new area column
    gscript.read_command('v.db.addcolumn',
                         map=map,
                         columns="area_square_meters DOUBLE PRECISION")
    
    #compute area and insert into area column
    gscript.read_command('v.to.db',
                         map=map,
                         type='centroid',
                         option='area',
                         columns='area_square_meters',
                         unit='meters')

create table showing total area by landcover type

In [11]:
def getLandcoverAreaByType(map):
    #get area data
    table = connectToAttributeTable(map=map)
    table.filters.select()
    columns = table.columns.names()
    cursor = table.execute()
    result = np.array(cursor.fetchall())
    cursor.close()
    data = pandas.DataFrame(result, columns=columns).set_index('cat')

    #sum areas by landcover type ('b_lbtyp')
    data['area_square_meters'] = pandas.to_numeric(data['area_square_meters'])
    areas = data[['b_lbtyp', 'area_square_meters']].groupby(by='b_lbtyp').sum()
    return areas

utility functions

In [12]:
def exportVectorToShapefile(map, output):
    gscript.read_command('v.out.ogr',
                         input=map,
                         format='ESRI_Shapefile',
                         output=output,
                         flags='e',
                         overwrite=True)

In [13]:
def getVectorMapInfo(map):
    return gscript.read_command('v.info', map=map)

In [14]:
#view(['1_viewshed'])
# TODO determine how to display vector map

#### Compute landcover areas

connect to 'sample_points_field' attribute table

In [15]:
point_table = connectToAttributeTable(map='sample_points_field')
point_table.filters.select()
columns = point_table.columns.names()
cursor = point_table.execute()
result = np.array(cursor.fetchall())
cursor.close()
point_data = pandas.DataFrame(result, columns=columns).set_index('cat')

loop through sample points

In [16]:
use_viewshed = False
viewshed_suffix = ''
with Vector('sample_points_field', mode='r') as points:
    
    #setup progress bar
    progress_bar = pyprind.ProgBar(points.n_lines, bar_char='█', title='Landcover analysis progress', monitor=True, stream=1, width=50)
    
    #iterate through points
    for point in points:
        
        #get point ID (SiteID)
        ID = point_data['ID'][point.cat-1]
        
        #update progress bar
        progress_bar.update(item_id=ID)
        
        #buffer current point
        extractPoint(input='sample_points_field', ID=ID, output='tmp_buffer_point')
        bufferPoint(input='tmp_buffer_point', output='tmp_point_buffer', radius=500)
        
        #set buffer as overlay
        overlay_input = 'tmp_point_buffer'
        #consider only visible landcover if 'use_viewshed' = True
        if use_viewshed:
            viewshed = 'vect_{0}_viewshed'.format(ID)
            visible_viewshed = 'vect_{0}_viewshed_{1}m'.format(ID, radius)
            #vectorize viewshed
            vectorizeViewshed(input='{0}_viewshed'.format(ID), ID=ID, output=viewshed)
            #overlay buffer on viewshed
            overlay(overlay=overlay_input,
                    underlay=viewshed,
                    output=visible_viewshed)
            #set overlay to the visible viewshed
            overlay_input = visible_viewshed
            #set suffix to be appended to filenames and descriptions
            viewshed_suffix = '_viewshed'
        overlay_output = 'vect_{0}_landcover_{1}m{2}'.format(ID, radius, viewshed_suffix)
        #overlay landcover
        overlay(overlay=overlay_input,
                         underlay='landcover',
                         output=overlay_output)
        
        #calculate landcover area
        calculateAreas(map=overlay_output)

Landcover analysis progress
0%                                     100%
[█████████████████████████████████████████] | ETA: 00:00:00 | Item ID: 111
Total time elapsed: 00:24:02


#### Export landcover area table

create table with landcover areas by type for each point

In [28]:
#create table
index_start = 41
n_types = 15  #number of landcover types
columns = ['ID', 'SiteID', 'IncludedArea']
columns = columns + [ str(n) for n in range(1,n_types+1) ]
area_table = pandas.DataFrame(columns=columns)

#set naming variables
viewshed_suffix = '_viewshed'
included_area = '{0}m{1}'.format(radius, viewshed_suffix)  #0=radius, 1=viewshed_suffix
map_pattern = 'vect_{0}_landcover_{1}m{2}'  #0=ID, 1=radius, 2=viewshed_suffix

#iterate through points
for index, point in point_data.iterrows():
    ID = point['ID']
    map = map_pattern.format(ID, radius, viewshed_suffix)
    #initiate row
    row = {'ID':"{0:.3g}".format(int(index) + index_start),
           'SiteID': str(ID),
           'IncludedArea': included_area}
    #get landcover areas
    areas = getLandcoverAreaByType(map)
    
    #iterate through area types
    for index, area in areas.iterrows():
        #add area to row
        row[index] = area['area_square_meters']
    #append row to table
    area_table = area_table.append(row, ignore_index=True)

area_table.set_index('ID', inplace=True)

#export table to file
area_table.to_csv('landcover_areas_{0}m{1}.csv'.format(radius, viewshed_suffix), header=False)